# Author: Johanna

We use this notebook to do a basic ensemble on different architectures. This is a simple average ensemble like introduced in <br/>
https://arxiv.org/abs/1911.06475 <br/>
and does not use any meta learner to combine predictions.

First, we do the usual setup.

In [ ]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

import os 
import tensorflow as tf
from pathlib import Path
import cv2

# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet
import numpy as np

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam, SGD
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, SINGLE_CLASS_METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, BinaryCrossentropy, compute_class_weight


In [ ]:
!remote_access/get_tunnels.sh

We specify our benchmark for the ensemble. All training for the single ensemble models is done here to ensure that the dataset split as well as all preprocessing and hyperparameters are the same for all ensemble members.

In [ ]:
columns_12 = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
columns_5 =  ['Cardiomegaly',
                'Edema',
                'Consolidation',
                'Atelectasis',
                'Pleural Effusion']

uzeros = ['Cardiomegaly',
        'Enlarged Cardiomediastinum',
        'Lung Opacity',
        'Lung Lesion',
        'Consolidation',
        'Pneumothorax',
        'Pleural Effusion']

uones = ['Edema',
        'Atelectasis',
        'Fracture',
        'Pleural Other',
        'Pneumonia',]

upsample_factors = {
    "Enlarged Cardiomediastinum": 1,
    "Lung Lesion":1,
    #"Pneumothorax":1,
    #"Pneumonia":1,
    "Pleural Other":2,
    "Fracture":2,
}
transformations_0 = {"hist_equalization":{}}
transformations_1 = { "gaussian_blur":{"kernal_size":3}, "hist_equalization":{}}
transformations_2 = {"unsharp_mask":{"radius":2, "amount":1}}
transformations_3 = {"windowing"}
transformations_4 = {}

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                     name="Chexpert_Masked_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled",
                                     classes=columns_12,
                                     #train_labels = "train.csv",
                                     train_labels = "nofinding_train.csv",
                                     test_labels = "test.csv",
                                     nan_replacement = -1, #float("NaN"),
                                     u_enc = [uzeros, uones],
                                     epochs=3,
                                     batch_size=32,
                                     crop = False,
                                     dim=(256, 256),
                                     loss = BinaryCrossentropy(),
                                     use_class_weights = False,
                                     upsample_factors = upsample_factors,
                                     metrics=METRICS,
                                     single_class_metrics=SINGLE_CLASS_METRICS,
                                     optimizer = Adam(learning_rate=1e-4, clipnorm=1),
                                     lr_factor = 0.1,
                                     augmentation = "affine",
                                     transformations = transformations_2,
                                     split_seed = 6122156,
                                     split_valid_size = 0.05, 
                                     preprocess_input_fn = tf.keras.applications.inception_resnet_v2.preprocess_input)


bce_benchmark.loss = WeightedBinaryCrossentropy(bce_benchmark.positive_weights,
                                                bce_benchmark.negative_weights)

bce_chexpert_exp = simple_architecture_experiment(bce_benchmark, InceptionResNetV2, bce_benchmark.label_columns)

In [ ]:
bce_chexpert_exp.benchmark.as_dict()

In [ ]:
bce_chexpert_exp.train()

In [ ]:
bce_chexpert_exp.evaluate()

In [ ]:
bce_chexpert_exp.save()

We load the individual members of the ensemble and do predictions on the test data generator. Those predictions are then averaged and evaluated with the sklearn classification report.

In [ ]:
predictions = []

In [ ]:
weights = "models/InceptionResNetV2_Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled/weights.05-0.21.hdf5"
bce_chexpert_exp.model.load_weights(weights)
predictions.append(bce_chexpert_exp.model.predict(bce_chexpert_exp.benchmark.testgen))

In [ ]:
len(predictions)

In [ ]:
from sklearn.metrics import classification_report

y_pred = tf.math.reduce_mean(predictions, axis=0)
predictions_bool = (y_pred >= 0.5)
groundtruth_label = bce_chexpert_exp.benchmark.testgen.get_labels_nonan()

for key, value in enumerate(columns_12):
    pr = []
    for sample in y_pred:
        pr.append(sample[key])
    m = tf.keras.metrics.AUC()
    m.update_state(groundtruth_label[:, key], pr)
    print(value, m.result().numpy())
    
m = tf.keras.metrics.AUC()
m.update_state(groundtruth_label, y_pred)
print("Average AUC: ", m.result().numpy())

predictions_bool = np.array(predictions_bool, dtype=int)
groundtruth_label = bce_chexpert_exp.benchmark.testgen.get_labels_nonan()
report = classification_report(groundtruth_label, predictions_bool, target_names=bce_chexpert_exp.benchmark.label_columns)
print('skelarn report: ', report)